# 2D SWFlow - Solitary wave over triangular shelf with island
------------------------------------------------------------------------------------------

This notebook uses Proteus to reproduce the 2009-2010 experiments of[Swigler, 2009] and [Lynett, 2019] performed at the O.H. Hinsdale Wave Research Laboratory of Oregon State University. The experiments were conducted
to study specific phenomena that are known to occur when solitary waves propagate over irregular bathymetry such as shoaling, refraction, breaking, etc. A video of the experiment can be seen here: [link](https://journals.tdl.org/icce/index.php/icce/article/view/1385/7701)

The 2D computational domain is defined as D = [0, 48.8m] x [-13.25, 13.25m]. The topography (interchanged with the bathymetry nomenclature) is defined in Section 7.5.3 of [Guermond et al, 2022]. A solitary wave is initiated at x = 5m and is assumed to be traveling to the right. The solitary wave will propagate towards the shelf and interact with the topography. The boundary conditions are all set to reflecting in this example.

This example tests the capabilities of handling dry states in the shallow water models implemented in PROTEUS. For more details, we refer to the user to the specific run file `reef_island_runup.py` or the references listed below.

### References

- Swigler, D.T., (2009) Investigating the Three-dimensional Turbulence and Kinematic Properties Associated with a     Breaking Solitary Wave. Master’s thesis, Texas A&M University., College Station, Texas. [URL](http://oaktrust.library.tamu.edu/bitstream/handle/1969.1/ETD-TAMU-2009-08-821/SWIGLER-THESIS.pdf?sequence=3)

- P. Lynett, D. Swigler, H. El Safty, L. Motoya, A. Keen, S. Son, and P. Higuera. Study of the three-dimensional       hydrodynamics associated with a solitary wave traveling over an alongshore-variable, shallow shelf. Journal of       Waterway, Port, Coastal, and Ocean Engineering (ASCE), 2019. [DOI: 10.1061/(ASCE)WW.1943-5460.0000525](https://ascelibrary.org/doi/abs/10.1061/%28ASCE%29WW.1943-5460.0000525)

# Running the benchmark via the terminal

The `parun` script can be to execute the python script file: `reef_island_runup.py`. There are several argument that can be supplied to the `parun` script to define various runtime options. All available options are listed when executing `parun -h` in the command line. Common command-line options are as follows:

**Option** | **Description**
:---: | :---:
 -v   | Print logging information to standard output
 -O PETSCOPTIONSFILE  | Text file of options to pass to Petsc library
 -D DATADIR | Set data directory for output storage
 -l LOGLEVEL | Store runtime information at the log level, 0 = none, 10 = everything
 -b BATCHFILENAME | Text file of auxiliary commands to execute along with main program
 -G gatherArchive | Collect data files into single file at end of simulation (will require more computational resources on large runs)
 -H hotStart | Use the last step in the archive as the initial condition and continue appending to the archive
 --SWEs | To consider SWEs applications
 
 
To run the script on more than one rank, one can invoke the following: `mpiexec -n <number of cores>` before the use of `parun` in the command line. 

## Context options for run file

Most (if not all) Proteus run files `benchmark_name.py` (in this case `reef_island_runup.py`) contain run time options specific to the model at hand. Here are some run time options for this particular example. For exact options, see the run file.

**Option** | **Description**
:---: | :---:
 sw_model | sw_model = {0,1} for {SWEs,DSWEs} 
 final_time  | Final time for simulation
 dt_output | Time interval to output solution
 mannings | Mannings roughness coefficient
 
 
To modify the context options at run time, include the `-C` flag followed by `"option1=True option2=2 ..."`.

In [ ]:
# Clean up previous data directory if it exists
!rm swflow_data/reef*

In [ ]:
# Then we run 
!mpiexec -np 2 parun --SWEs reef_island_runup.py -v -l1 -C "final_time=20. sw_model=1 mannings=1." -D run_data

## Post-process the solution using ipygany

In [ ]:
# Get dependencies
import sys
sys.path.append('../../../proteus_visualization')
from hdf5_loader import extract_arrays_metadata, extract_array
import numpy as np
from ipywidgets import Image
from ipywidgets import Play, IntSlider, HBox, link
from ipygany import Scene, Data, Component, PolyMesh, Water, UnderWater, Data, Component, Threshold
from ipydatawidgets import NDArrayWidget

In [ ]:
# Load our data
arrays_metadata = extract_arrays_metadata('./run_data/reef_island_runup.h5')

mem_vertices = extract_array(arrays_metadata, 'nodesSpatial_Domain0')
vertices = np.array(mem_vertices[:, 0:2])

indices = extract_array(arrays_metadata, 'elementsSpatial_Domain0')

# This never changes, we extract it only once
bathymetry = extract_array(arrays_metadata, 'bathymetry0_t0')

# Get texture for topography
texture = Image.from_file('./cement.jpg')

In [ ]:
# Define simulation parameters
warp_value = 5.
num_of_steps = 200

In [ ]:
# Caching arrays on the front-end using NDArrayWidgets
h_cached = []
water_vertices_cached = []
for i in range(num_of_steps):
    h = extract_array(arrays_metadata, 'h_t{}'.format(i))

    z_water = h + bathymetry
    water_vertices = np.append(vertices, z_water.reshape((z_water.shape[0], 1)) * warp_value, axis=1).flatten()

    h_cached.append(NDArrayWidget(array=h))
    water_vertices_cached.append(NDArrayWidget(array=water_vertices))   

In [ ]:
# Set up ipygany for visualizing the solution 

h_component = Component(name='h', array=h_cached[0])

water_mesh = PolyMesh(
    vertices=water_vertices_cached[0],
    triangle_indices=indices,
    data={'h': [h_component]}
)

actual_water = Threshold(water_mesh, input='h', min=1e-3, max=1000)

floor = PolyMesh(
    vertices=np.append(vertices, bathymetry.reshape((bathymetry.shape[0], 1)) * warp_value, axis=1),
    triangle_indices=indices,
    data={'underwater': [h_component]}
)

water = Water(
    actual_water, 
    under_water_blocks=(UnderWater(floor), ),
    caustics_enabled=True
)

scene = Scene((water, ))

def update_step(change):
    i = change['new']

    h_component.array = h_cached[i]
    water_mesh.vertices = water_vertices_cached[i]

play = Play(description='Step:', min=0, max=num_of_steps-1, value=0, interval=100)
play.observe(update_step, names=['value'])

progress = IntSlider(value=0, step=1, min=0, max=num_of_steps-1)
link((progress, 'value'), (play, 'value'))

display(HBox((play, progress)))

# Visualize solution 
scene

In [ ]:
# Define some visualization parameters
#water.caustics_factor = 0.20
#water.under_water_blocks[0].texture = texture
#scene.background_color='aliceblue'